# Working on a Baseline

In [2]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
from bs4 import BeautifulSoup
import requests
import json
import time
import string
# from tqdm import *
import datetime
import math

In [73]:
#############################################################################
## READ IN THE CSVs WITH ALL THE DATA AND REDO THE DATETYPE TRANSFORMATION ## 
#############################################################################
traindf = pd.read_csv('/Users/william/DropBox/CS 109/traindf.csv', index_col=0);
traindf.date = pd.to_datetime(traindf.date)
traindf['name'] = traindf['name'].astype(str)
traindf['name'] = map(lambda x: x.upper(), traindf['name'])


testdf = pd.read_csv('tempdata/testdf.csv', index_col=0);
testdf.date = pd.to_datetime(testdf.date)
testdf['name'] = testdf['name'].astype(str)
testdf['name'] = map(lambda x: x.upper(), testdf['name'])
testdf[['win', 'place', 'show']].fillna(0, inplace=True)
testdf = testdf[testdf.date >= datetime.datetime(1999,1,1)]

In [74]:
traindf = traindf[traindf.name != 'NAN']
traindf.reset_index().drop('index', axis=1, inplace=True);

In [163]:
testdf.head()

,post,name,win,place,show,fin_pos,Odds,date,track,fog,hail,maxhumidity,meandewptm,meanpressurem,meantempm,meanvism,meanwdird,meanwindspdm,minhumidity,precipm,rain,snow,snowdepthm,snowfallm,thunder
3,6,CHARISMATIC,18.8,7.6,5.8,1,8.4,1999-05-15,PIM,0,0,64,2,1025,16,16,75,14,24,0.00,0,0,NaN,0,0
4,5,MENIFEE,0.0,3.6,3.2,2,2.0,1999-05-15,PIM,0,0,64,2,1025,16,16,75,14,24,0.00,0,0,NaN,0,0
5,4,BADGE,0.0,0.0,18.8,3,58.0,1999-05-15,PIM,0,0,64,2,1025,16,16,75,14,24,0.00,0,0,NaN,0,0
6,4,RED BULLET,14.4,3.2,2.8,1,6.2,2000-05-20,PIM,0,0,100,12,1017,12,6,65,18,93,0.76,1,0,NaN,0,0
7,7,FUSAICHI PEGASUS,0.0,2.6,2.2,2,0.3,2000-05-20,PIM,0,0,100,12,1017,12,6,65,18,93,0.76,1,0,NaN,0,0


In [44]:
traindf.head()

,track,racenum,date,post,name,win,fin_pos,place,show,fog,hail,maxhumidity,meandewptm,meanpressurem,meantempm,meanvism,meanwdird,meanwindspdm,minhumidity,precipm,rain,snow,snowdepthm,snowfallm,thunder
0,AQU,1,1998-10-28,3,CURVY IMAGE,6.3,1,3.8,3.0,0,0,96,11,1013,12,15,195,13,75,4.06,1,0,NaN,0,0
1,AQU,2,1998-10-28,3,MILLIONDOLLARSMILE,14.0,1,4.8,3.1,0,0,96,11,1013,12,15,195,13,75,4.06,1,0,NaN,0,0
2,AQU,3,1998-10-28,6,ROOK,6.5,1,3.4,2.4,0,0,96,11,1013,12,15,195,13,75,4.06,1,0,NaN,0,0
3,AQU,4,1998-10-28,2,WAPO,6.2,1,5.1,2.8,0,0,96,11,1013,12,15,195,13,75,4.06,1,0,NaN,0,0
4,AQU,5,1998-10-28,8,FUNNY TOY (ARG),16.0,1,7.7,3.5,0,0,96,11,1013,12,15,195,13,75,4.06,1,0,NaN,0,0
5,AQU,6,1998-10-28,6,RUBY RUBLES,11.4,1,5.6,4.8,0,0,96,11,1013,12,15,195,13,75,4.06,1,0,NaN,0,0
6,AQU,7,1998-10-28,5,EXCHANGE PLACE,9.2,1,4.4,2.9,0,0,96,11,1013,12,15,195,13,75,4.06,1,0,NaN,0,0
7,AQU,8,1998-10-28,2,EARLY WARNING,16.4,1,6.6,5.3,0,0,96,11,1013,12,15,195,13,75,4.06,1,0,NaN,0,0
8,AQU,9,1998-10-28,4,JET BLACK,20.4,1,11.0,6.4,0,0,96,11,1013,12,15,195,13,75,4.06,1,0,NaN,0,0
9,AQU,1,1998-10-29,1a,CRITICS ACCLAIM,3.6,1,2.2,2.1,0,0,90,3,1011,11,16,325,21,38,0.00,0,0,NaN,0,0


In [49]:
# http://stackoverflow.com/questions/1249388/how-do-we-remove-all-non-numeric-characters-from-a-string-in-python
def only_numerics(seq):
    seq_type= type(seq)
    return seq_type().join(filter(seq_type.isdigit, seq))

'17'

In [132]:
names = xtest.name.unique()
xtrain = pd.DataFrame(columns=traindf.columns)
for name in names:
    temp = traindf[traindf.name == name]
    xtrain = pd.concat([xtrain, temp])
xtrain.head()

,track,racenum,date,post,name,win,fin_pos,place,show,fog,hail,maxhumidity,meandewptm,meanpressurem,meantempm,meanvism,meanwdird,meanwindspdm,minhumidity,precipm,rain,snow,snowdepthm,snowfallm,thunder
96446,CHU,8,1999-05-01,11,CHARISMATIC,64.6,1,27.8,14.4,0,0,77,6,1021,17,16,17,13,29,0,0,0,NaN,0,0
96538,CHU,9,1999-05-15,6,CHARISMATIC,18.6,1,NaN,NaN,0,0,93,14,1018,20,13,110,10,54,0,0,0,NaN,0,0
365529,HPX,1,1998-11-21,3,CHARISMATIC,4.6,1,2.4,2.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
396465,KND,8,1999-04-18,5,CHARISMATIC,26.2,1,10.2,6.2,0,0,89,3,1022,8,16,233,11,46,0,0,0,NaN,0,0
706750,SUF,9,1999-04-18,5,CHARISMATIC,12.6,1,NaN,NaN,0,0,76,2,1006,12,16,273,23,34,0,0,0,NaN,0,0


In [134]:
notcols_train = ["racenum", "place", "show", "fin_pos"]
temp_train = list(set(traindf.columns) - set(notcols_train))
notcols_test = ["place", "show"]
temp_test = list(set(testdf.columns) - set(notcols_test))

xtrain = xtrain[temp_train]
xtrain = xtrain.reset_index().drop('index', axis=1)
excepts = []
for index in xtrain.index:
    try:
        xtrain["post"].loc[index] = only_numerics(xtrain["post"].loc[index])
    except Exception, e:
        excepts.append(str(e))
        xtrain["post"].loc[index] = 0.0
ytrain = xtrain["win"]

xtest = testdf[temp_test]
xtest = xtest.reset_index().drop('index', axis=1)
excepts1 = []
for index in xtest.index:
    try:
        xtest["post"].loc[index] = only_numerics(xtest["post"].loc[index])
    except Exception, e:
        excepts1.append(str(e))
        xtest["post"].loc[index] = 0.0
ytest = testdf["win"]

posts = xtrain.post.unique()
tracks = xtrain.track.unique()

#make a column for each genre and fill with true or false
for post in posts:
    xtrain[post] = [post == row for row in xtrain.post]
    xtest[post] = [post == row for row in xtest.post]

for track in tracks:
    xtrain[track] = [track == row for row in xtrain.track]
    xtest[track] = [track == row for row in xtest.track]

xtrain.head()

,minhumidity,maxhumidity,meanwindspdm,meanpressurem,meandewptm,win,snow,hail,precipm,meanvism,track,snowfallm,fog,meanwdird,rain,snowdepthm,thunder,date,post,meantempm,name,11,6,3,5,4,2,7,13,10,1,12,8,9,14,15,20,16,19,18,17,CHU,HPX,KND,SUF,BEL,MON,SAN,SGA,LDN,AQU,DPK,GUL,IND,CAL,HOO,OAK,DEL,MDL,PIM,SPT,TUR,FLX,LON,ARL,LAU,THI,FLK,FGD,PHI,FMT,PRM,HAW,TAM,RET,REM,MTR,PAR,DDN,COL,BAY,GOL,BEU,ELL,KEN,SAM,GLD,PEN,CTN,LOS,TIM,CAN,EVN,EMD,RIV
0,29,77,13,1021,6,64.6,0,0,0,16,CHU,0,0,17,0,NaN,0,1999-05-01,11,17,CHARISMATIC,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,54,93,10,1018,14,18.6,0,0,0,13,CHU,0,0,110,0,NaN,0,1999-05-15,6,20,CHARISMATIC,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,NaN,NaN,NaN,NaN,NaN,4.6,NaN,NaN,NaN,NaN,HPX,NaN,NaN,NaN,NaN,NaN,NaN,1998-11-21,3,NaN,CHARISMATIC,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,46,89,11,1022,3,26.2,0,0,0,16,KND,0,0,233,0,NaN,0,1999-04-18,5,8,CHARISMATIC,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,34,76,23,1006,2,12.6,0,0,0,16,SUF,0,0,273,0,NaN,0,1999-04-18,5,12,CHARISMATIC,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [77]:
#########################
### OUT OF COMMISSION ###
#########################
from sklearn.linear_model import LinearRegression
def do_regression(xtr, ytr, date):
    if len(xtr) <= 1:
        return np.mean(xtrain.loc[xtrain.date <= date, "win"])
    else:
        x.drop(["name", "win", "track", "post", "date"], inplace=True, axis=1)
        lm = LinearRegression()
        return lm.fit(x,y)
    
regdictlist = []
for horse in names:
    reg_dict = {}
    reg_xtrain = xtrain[xtrain.name == horse]
    reg_ytrain = ytrain[xtrain.name == horse]
    reg_xtrain = reg_xtrain.fillna(0)
    reg_ytrain = reg_ytrain.fillna(0)
    if len(reg_xtrain) > 0:
        for d in reg_xtrain.date:
            track = reg_xtrain.loc[(reg_xtrain.date == d) & (reg_xtrain.name == horse), "track"].iloc[0]
            reg_dict[(d, track, horse)] = do_regression(reg_xtrain, reg_ytrain, d)
    regdictlist.append(reg_dict)
    
train_pred = {}
test_pred = {}
for item in regdictlist:
    for key in item.keys():
        model = item[key]
        pred_dict = {}
        if not isinstance(model, float):
            train_row = xtrain.loc[(xtrain.date == key[0]) & (xtrain.name == key[2]) & (xtrain.track == key[1])]
            train_row = train_row.drop(["name", "win", "track", "post", "date"], axis=1)
            train_row = train_row.fillna(0)
            pred_dict["win"] = model.predict(train_row)[0]
            train_pred[key] = pred_dict
        else:
            pred_dict["win"] = model
            train_pred[key] = pred_dict
        
        #test_row = xtest.loc[xtest.date == key[0]]
        #test_row = test_row.drop(["name", "win", "track", "post", "date"], axis=1)
        #test_row = test_row.fillna(0)
        #testpred[key] = model.predict(test_row)

In [159]:
print  set(xtest.columns) - set(xtrain.columns)

set(['Odds', 'fin_pos'])


In [161]:
def find_races(horses_name, traindf=traindf):
    return traindf[traindf['name']==horses_name]

test_pred = {}
for i in xtest.index:
    pred_dict = {}
    
    horse = xtest.loc[i,"name"]
    d = xtest.loc[i, "date"]
    track = xtest.loc[i, "track"]
    key = (d, track, horse)
    
    races = find_races(horse, xtrain)
    reg_xtrain = races.loc[races.date < d]
    reg_ytrain = reg_xtrain["win"]
    reg_xtrain = reg_xtrain.fillna(0)
    reg_ytrain = reg_ytrain.fillna(0)
    
    if len(reg_xtrain) <= 1:
        pred_dict["win"] = np.mean(xtrain.loc[xtrain.date <= d, "win"])
        train_pred[key] = pred_dict
    else:
        reg_xtrain = reg_xtrain.drop(["name", "win", "track", "post", "date"], axis=1)
        lm = LinearRegression()
        reg_fit = lm.fit(reg_xtrain, reg_ytrain)
        row = xtest.loc[[i]].drop(["name", "win", "track", "post", "date", "Odds", "fin_pos"], axis=1).fillna(0)
        
        pred_dict["win"] = reg_fit.predict(row)[0]
        test_pred[key] = pred_dict

In [162]:
test_pred

{(Timestamp('1999-05-01 00:00:00'),
  'CHU',
  'ANSWER LIVELY'): {'win': 5.5568848559258441},
 (Timestamp('1999-05-01 00:00:00'),
  'CHU',
  'CAT THIEF'): {'win': 201.17340397397223},
 (Timestamp('1999-05-01 00:00:00'),
  'CHU',
  'CHARISMATIC'): {'win': 2.7429166390835995},
 (Timestamp('1999-05-01 00:00:00'),
  'CHU',
  'ECTON PARK'): {'win': -33.680701754385964},
 (Timestamp('1999-05-01 00:00:00'),
  'CHU',
  'EXCELLENT MEETING'): {'win': 7.0508668723015688},
 (Timestamp('1999-05-01 00:00:00'),
  'CHU',
  'K ONE KING'): {'win': -65.308621652005399},
 (Timestamp('1999-05-01 00:00:00'),
  'CHU',
  'KIMBERLITE PIPE'): {'win': 155.53150882685762},
 (Timestamp('1999-05-01 00:00:00'),
  'CHU',
  'LEMON DROP KID'): {'win': 48.801749979617362},
 (Timestamp('1999-05-01 00:00:00'),
  'CHU',
  'PRIME TIMBER'): {'win': 2.8275250983287665},
 (Timestamp('1999-05-01 00:00:00'),
  'CHU',
  'VICAR'): {'win': 210.00577800408433},
 (Timestamp('1999-05-15 00:00:00'),
  'PIM',
  'BADGE'): {'win': -3.8740

In [145]:
xtest.iloc[[1]]

,win,hail,snowdepthm,thunder,snow,meanvism,track,fog,fin_pos,meanwindspdm,meanpressurem,meandewptm,Odds,rain,minhumidity,meanwdird,date,post,maxhumidity,name,precipm,snowfallm,meantempm,11,6,3,5,4,2,7,13,10,1,12,8,9,14,15,20,16,19,18,17,CHU,HPX,KND,SUF,BEL,MON,SAN,SGA,LDN,AQU,DPK,GUL,IND,CAL,HOO,OAK,DEL,MDL,PIM,SPT,TUR,FLX,LON,ARL,LAU,THI,FLK,FGD,PHI,FMT,PRM,HAW,TAM,RET,REM,MTR,PAR,DDN,COL,BAY,GOL,BEU,ELL,KEN,SAM,GLD,PEN,CTN,LOS,TIM,CAN,EVN,EMD,RIV
1,0,0,NaN,0,0,16,PIM,0,2,14,1025,2,2,0,24,75,1999-05-15,0,64,MENIFEE,0,0,16,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [84]:
regdictlist[:1]

[{(Timestamp('1998-10-01 00:00:00'), 'SAN', 'CHARISMATIC'): 9.466666666666667,
  (Timestamp('1998-11-21 00:00:00'),
   'HPX',
   'CHARISMATIC'): LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False),
  (Timestamp('1998-12-27 00:00:00'),
   'SAN',
   'CHARISMATIC'): LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False),
  (Timestamp('1999-04-18 00:00:00'),
   'KND',
   'CHARISMATIC'): LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False),
  (Timestamp('1999-05-01 00:00:00'),
   'CHU',
   'CHARISMATIC'): LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False),
  (Timestamp('1999-05-15 00:00:00'),
   'CHU',
   'CHARISMATIC'): LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False),
  (Timestamp('1999-06-05 00:00:00'),
   'BEL',
   'CHARISMATIC'): LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)}]

In [128]:
testpred_dict = {}
ddates = []
for key in train_pred.keys():
    ddates.append(key[0])
    if (key[0] in xtest.date.unique()):
        print 'yes'
        if (key[1] in ["CHU", "PIM", "BEL"]):
            testpred_dict[key] = train_pred[key]
for each in sorted(ddates):
    print each

1998-08-15 00:00:00
1998-08-21 00:00:00
1998-08-21 00:00:00
1998-08-21 00:00:00
1998-08-29 00:00:00
1998-09-11 00:00:00
1998-09-13 00:00:00
1998-09-20 00:00:00
1998-09-20 00:00:00
1998-09-22 00:00:00
1998-09-30 00:00:00
1998-10-01 00:00:00
1998-10-03 00:00:00
1998-10-10 00:00:00
1998-10-10 00:00:00
1998-10-11 00:00:00
1998-10-14 00:00:00
1998-10-14 00:00:00
1998-10-17 00:00:00
1998-10-18 00:00:00
1998-10-18 00:00:00
1998-10-18 00:00:00
1998-10-24 00:00:00
1998-11-01 00:00:00
1998-11-04 00:00:00
1998-11-07 00:00:00
1998-11-07 00:00:00
1998-11-07 00:00:00
1998-11-07 00:00:00
1998-11-07 00:00:00
1998-11-12 00:00:00
1998-11-18 00:00:00
1998-11-18 00:00:00
1998-11-21 00:00:00
1998-11-22 00:00:00
1998-11-25 00:00:00
1998-11-28 00:00:00
1998-11-28 00:00:00
1998-11-28 00:00:00
1998-11-28 00:00:00
1998-12-03 00:00:00
1998-12-04 00:00:00
1998-12-06 00:00:00
1998-12-08 00:00:00
1998-12-12 00:00:00
1998-12-12 00:00:00
1998-12-12 00:00:00
1998-12-12 00:00:00
1998-12-13 00:00:00
1998-12-13 00:00:00


In [123]:
print xtest.date.unique()

['1999-05-14T20:00:00.000000000-0400' '2000-05-19T20:00:00.000000000-0400'
 '2001-05-18T20:00:00.000000000-0400' '2002-05-17T20:00:00.000000000-0400'
 '2003-05-16T20:00:00.000000000-0400' '2004-05-14T20:00:00.000000000-0400'
 '2005-05-20T20:00:00.000000000-0400' '2006-05-19T20:00:00.000000000-0400'
 '2007-05-18T20:00:00.000000000-0400' '2008-05-16T20:00:00.000000000-0400'
 '2009-05-15T20:00:00.000000000-0400' '2010-05-14T20:00:00.000000000-0400'
 '2011-05-20T20:00:00.000000000-0400' '2012-05-18T20:00:00.000000000-0400'
 '2013-05-17T20:00:00.000000000-0400' '2014-05-16T20:00:00.000000000-0400'
 '2015-05-15T20:00:00.000000000-0400' '1999-04-30T20:00:00.000000000-0400'
 '2000-05-05T20:00:00.000000000-0400' '2001-05-04T20:00:00.000000000-0400'
 '2002-05-03T20:00:00.000000000-0400' '2003-05-02T20:00:00.000000000-0400'
 '2004-04-30T20:00:00.000000000-0400' '2005-05-06T20:00:00.000000000-0400'
 '2006-05-05T20:00:00.000000000-0400' '2007-05-04T20:00:00.000000000-0400'
 '2008-05-02T20:00:00.000

In [121]:
len(testpred_dict.keys())

0

In [ ]:
def find_races(horses_name, traindf=traindf):
    return traindf[traindf['name']==horses_name]

def find_avg_payouts(df, race_date=datetime.date.today()):
    out = {}
    df = df[df.date < race_date]
    out['win'] = np.mean(df.win[df.win != 0.0])
    out['place'] = np.mean(df.place[df.place != 0.0])
    out['show'] = np.mean(df.show[df.show != 0.0])
    return out

find_avg_payouts(find_races('SMARTY JONES', traindf=testdf), race_date=datetime.datetime(1998, 12, 12))

The payouts are already standardized to $2 bets, so we luckily don't have to deal with that problem

In [ ]:
unique_test_dates = testdf['date'].unique()

def find_horses(date):
    return testdf['name'].loc[testdf['date'] == date]

def find_track(date):
    return testdf['track'].loc[testdf['date'] == date][0]

#find_horses(datetime.datetime(1998, 5, 15))
#type(testdf['date'][0])

We run a simple regression (aka Average) to get a dict where the keys are a tuple `(date, track, horse)` and the values are themselves dictionaries with keys `win`, `place`, `show` and values as the corresponding averages leading into raceday (excluding information we couldn't have on raceday). 

In [ ]:
avg_payouts = {}
for i in testdf['date'].unique():
    race = testdf.loc[testdf.date == i]
    for j in range(len(race)):
        date = race.date.iloc[j]
        track = race.track.iloc[j]
        horse = race.name.iloc[j]
        avg_payouts[(date, track, horse)] = find_avg_payouts(
            find_races(race['name'].iloc[j], traindf=traindf), race_date=date)

In [ ]:
## this function takes a payout and returns the first odds digit, assuming a comparison to 1, i.e. x-1 
## of that horse to win when betting occured assuming original bet of $2 and assumed take by race track of 15% 
def payoff_to_odds(payoff, bet_amount=2.0, take = .15):
    return round(((payoff/(1-take) - bet_amount)/bet_amount),4)

def odds_to_percent(odds): 
    return (float(str(odds)[2]))/(float(str(odds)[0])+float((str(odds)[2])))

def normalize_odds(odds): 
    x = odds.split("-")
    if len(x) > 1: 
        return float(x[0])/float(x[1])
    else: 
        return float(x[0])
    
def make_favorite(string): 
    if "favorite" in string: 
        return True 
    else:
        return False 

In [ ]:
def baseline(indict):
    profit = []
    cost = []
    count = []
    count2 = []
    for key in indict.keys():
        count2.append(1)
        date = key[0]
        track = key[1]
        horse = key[2]
        
        exp_win = indict[key]['win']
        exp_place = indict[key]['place']
        exp_show = indict[key]['show']
        
        # get the morning line odds for that day to compare
        odds = testdf.loc[(testdf.name == horse) & (testdf.track == track) & (testdf.date == date)]['Odds'].iloc[0]

        if odds > payoff_to_odds(exp_win):
            cost.append(2)
            
            if math.isnan(testdf.loc[(testdf.name == horse) & 
                                     (testdf.track == track) & 
                                     (testdf.date == date)]['win'].iloc[0]):
                count.append(1)
                pass
            
            else:
                profit.append(testdf.loc[(testdf.name == horse) & 
                                     (testdf.track == track) & 
                                     (testdf.date == date)]['win'].iloc[0])
        else:
            count.append(1)
            pass
        
    return ['baseline', no_bets_placed, race, year, profit_wins, profit_places, profit_shows, total_profit, cost]
            
baseline(avg_payouts)

Dataframe of race, year, profit/loss on win baseline, profit/loss on place baseline, profit/loss on show baseline

In [ ]:
def make_bets_baseline(indict, unique_test_dates=unique_test_dates): 
    outdict = {}
    outdict['method'] = []
    outdict['no_bets_placed'] = []
    outdict['race'] = []
    outdict['year'] = []
    outdict['profit_wins'] = []
    outdict['profit_places'] = []
    outdict['profit_shows'] = []
    outdict['total_profit'] = []
    outdict['cost'] = []
    for unique_date in unique_test_dates:
        newdict = {}
        for key in indict.keys():
            date = key[0]
            track = key[1]
            horse = key[2]
            
            
            if date == unique_date:
                newdict[key] = indict[key]
                  
        # now we have newdict, subsetting based on race (by unique date)
        
        spread_tup = []
        for key in newdict.keys():
            date = key[0]
            track = key[1]
            horse = key[2]

            exp_win = newdict[key]['win']
            exp_place = newdict[key]['place']
            exp_show = newdict[key]['show']

            # get the morning line odds for that day to compare
            odds = testdf.loc[(testdf.name == horse) & 
                              (testdf.track == track) & 
                              (testdf.date == date)]['Odds'].iloc[0]
            
            spread = odds - payoff_to_odds(exp_win)
            spread_tup.append((horse, spread))
            
        sorted_list = sorted(spread_tup,key=lambda x: x[1], reverse=True)
        
        print sorted_list[0:3]
            
#         outdict['method'] = []
#         outdict['no_bets_placed'] = []
#         outdict['race'] = []
#         outdict['year'] = []
#         outdict['profit_wins'] = []
#         outdict['profit_places'] = []
#         outdict['profit_shows'] = []
#         outdict['total_profit'] = []
#         outdict['cost'] = []

make_bets_baseline(avg_payouts)